In [2]:
import pandas as pd
import numpy as np
import re
import math

%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
start = ['sproul', 'rieber', 'deneve', 'hedrick']
stop = ['target', 'court', 'powell', 'anderson', 'public', 'yrl']
path = ['A', 'B']
route_names = []

for a in range (0, 4):
    for b in range (0, 5):
        route_names.append(start[a] + '_' + stop[b] + '_A')
        route_names.append(start[a] + '_' + stop[b] + '_B')
print(route_names)
len(route_names)

['sproul_target_A', 'sproul_target_B', 'sproul_court_A', 'sproul_court_B', 'sproul_powell_A', 'sproul_powell_B', 'sproul_anderson_A', 'sproul_anderson_B', 'sproul_public_A', 'sproul_public_B', 'rieber_target_A', 'rieber_target_B', 'rieber_court_A', 'rieber_court_B', 'rieber_powell_A', 'rieber_powell_B', 'rieber_anderson_A', 'rieber_anderson_B', 'rieber_public_A', 'rieber_public_B', 'deneve_target_A', 'deneve_target_B', 'deneve_court_A', 'deneve_court_B', 'deneve_powell_A', 'deneve_powell_B', 'deneve_anderson_A', 'deneve_anderson_B', 'deneve_public_A', 'deneve_public_B', 'hedrick_target_A', 'hedrick_target_B', 'hedrick_court_A', 'hedrick_court_B', 'hedrick_powell_A', 'hedrick_powell_B', 'hedrick_anderson_A', 'hedrick_anderson_B', 'hedrick_public_A', 'hedrick_public_B']


40

In [77]:
class route():
    def __init__(self, num):        
        routes = ['sproul_target_A', 'sproul_target_B', 'sproul_court_A', 'sproul_court_B', 'sproul_powell_A', 'sproul_powell_B', 'sproul_anderson_A', 'sproul_anderson_B', 'sproul_public_A', 'sproul_public_B', 'rieber_target_A', 'rieber_target_B', 'rieber_court_A', 'rieber_court_B', 'rieber_powell_A', 'rieber_powell_B', 'rieber_anderson_A', 'rieber_anderson_B', 'rieber_public_A', 'rieber_public_B', 'deneve_target_A', 'deneve_target_B', 'deneve_court_A', 'deneve_court_B', 'deneve_powell_A', 'deneve_powell_B', 'deneve_anderson_A', 'deneve_anderson_B', 'deneve_public_A', 'deneve_public_B', 'hedrick_target_A', 'hedrick_target_B', 'hedrick_court_A', 'hedrick_court_B', 'hedrick_powell_A', 'hedrick_powell_B', 'hedrick_anderson_A', 'hedrick_anderson_B', 'hedrick_public_A', 'hedrick_public_B']
        self.route_name = routes[num]
        self.time = []
        self.total_calories = []
        self.descent = 0
        self.ascent = 0
        self.route_info = []
        
        route_gpx = self.route_name + '.gpx'
        route_csv = self.route_name + '.csv'
        
        lat = []
        lon = []
        ele = []
        
        with open(route_gpx, 'r') as f:
            y = f.read()
        for lat_v in re.findall(r'\slat="([\-\d\.]+)"', y):
            lat.append(math.radians(float(lat_v)))
        for long_v in re.findall(r'\slon="([\-\d\.]+)"', y):
            lon.append(math.radians(float(long_v)))
        for ele_v in re.findall(r'<ele>([\-\d\.]+)</ele>', y):
            ele.append(float(ele_v))

        
        #inverse haversine
        r = 3958.8 #miles (radius of earth)
        distance = [0]
        for a in range (0, len(ele)-1):
            q = math.sin( (lat[a+1] - lat[a]) / 2)**2
            w = math.cos(lat[a])*math.cos(lat[a+1])*math.sin( (lon[a+1] - lon[a]) / 2)**2
            d = 2 * r * math.asin(math.sqrt( q + w )) 
            distance.append(d)
        for a in range (0, len(distance)-1):
            distance[a+1] += distance[a]
        self.total_distance = (distance[len(distance)-1])
        
            
        self.total_distance_m = self.total_distance * 1609.34
        self.route_info.append(self.total_distance)
        self.route_info.append(self.total_distance_m)

        
        for a in range (0, len(ele)-1):
            temp = ele[a+1] - ele[a]
            if temp < 0:
                self.descent += temp * (-1)
            else:
                self.ascent += temp
        
        self.route_info.append(self.ascent)
        self.route_info.append(self.descent)


        distance_m = []
        for a in range (0, len(distance)):
            distance_m.append(distance[a] * 1609.34)
        gradient = []
        avg_distance = []
        for a in range (0, len(ele)-1):
            rise = ele[a+1] - ele[a]
            run = distance_m[a+1] - distance_m[a]
            avg_distance.append( (distance[a+1] + distance[a]) / 2)
            gradient.append(rise / run)
        gradient_data = pd.DataFrame(list(zip(avg_distance, gradient)), columns = ['avg_distance', 'gradient'])

        speed = [[.9, 'slow'], [1.35, 'normal'], [1.8, 'fast']]

        for a in range (0, 3):
            calories = []
            i = 1.44
            j = 1.94 * speed[a][0] ** 0.43
            k = 0.24 * speed[a][0] ** 4
            for b in range (0, len(gradient)):
                ee = i + j + k + 0.34 * speed[a][0] * gradient[b] * (1 - 1.05 **(1 - 1.1 ** (gradient[b] + 32)))
                calories.append(ee * (68.0389) * ((distance_m[b+1]-distance_m[b]) / speed[a][0]) / 4.184 / 1000)
            self.total_calories.append(sum(calories))
            self.time.append(self.total_distance_m / speed[a][0] / 60)
            
            self.route_info.append(self.total_calories[a])
            self.route_info.append(self.time[a])


In [78]:
data = pd.DataFrame(index=(route_names), columns=('distance(mi)', 'distance(m)', 'ascent(m)', 'descent(m)', 'slowcalories', 'slowtime', 'normalcalories', 'normaltime', 'fastcalories', 'fasttime'))
for x in range (0, 40):
    a = route(x)
    for b in range (0, 10):
        data.iloc[x][b] = a.route_info[b]
data.to_csv('route_info0.csv')
data

,distance(mi),distance(m),ascent(m),descent(m),slowcalories,slowtime,normalcalories,normaltime,fastcalories,fasttime
sproul_target_A,1.0942,1760.95,7.8,49,109.678,32.6101,94.1317,21.7401,102.586,16.3051
sproul_target_B,1.16626,1876.9,8.5,51.3,116.904,34.7575,100.334,23.1717,109.345,17.3787
sproul_court_A,0.806562,1298.03,21.4,34.6,80.9059,24.0376,69.4462,16.0251,75.6784,12.0188
sproul_court_B,0.779059,1253.77,16.6,29.8,78.1453,23.218,67.0763,15.4787,73.096,11.609
sproul_powell_A,0.618548,995.455,12.4,33.7,62.0079,18.4343,53.2195,12.2896,57.9989,9.21717
sproul_powell_B,0.690902,1111.9,10.6,29.9,69.2762,20.5907,59.4598,13.7271,64.7983,10.2953
sproul_anderson_A,0.499473,803.822,6.2,33.7,50.0354,14.8856,42.9388,9.92373,46.7982,7.4428
sproul_anderson_B,0.744391,1197.98,3.4,30.1,74.6189,22.1848,64.0426,14.7898,69.7944,11.0924
sproul_public_A,0.786937,1266.45,32.6,33.7,78.9783,23.4528,67.7974,15.6352,73.878,11.7264
sproul_public_B,0.987702,1589.55,22.2,31.2,99.1003,29.4361,85.0669,19.6241,92.6987,14.718


In [72]:
a = [0,1,2]
b = a[::-1]
for i in range (0, len(a)):
    b[i] -= 5
    
print(b)

[-3, -4, -5]
